In [1]:
import os
import matplotlib.pyplot as plt
import json
import numpy as np
from sklearn.decomposition import PCA
import pickle

import sys
sys.path.insert(0, '../')
from forest3D import lidar_IO,treeDetector,ground_removal,utilities,detection_tools
from forest3D.pcd import read_pcd
from forest3D import detection_tools,processLidar
from forest3D import lidar_IO,treeDetectorYolov3 as treeDetector,ground_removal,utilities,detection_tools
from forest3D.object_detectors import detectObjects_yolov3 as detectObjects
import matplotlib.pyplot as plt


/3d_forest/miniconda3/envs/3d_forest/lib/python3.9/site-packages/torch/hub.py:286: UserWarning: You are about to download and run code from an untrusted repository. In a future release, this won't be allowed. To add the repository to your trusted list, change the command to {calling_fn}(..., trust_repo=False) and a command prompt will appear asking for an explicit confirmation of trust, or load(..., trust_repo=True), which will assume that the prompt is to be answered with 'yes'. You can also use load(..., trust_repo='check') which will only prompt for confirmation if the repo is not already trusted. This will eventually be the default behaviour
  warnings.warn(
Downloading: "https://github.com/ultralytics/yolov5/zipball/master" to /root/.cache/torch/hub/master.zip
requirements: Ultralytics requirement "gitpython>=3.1.30" not found, attempting AutoUpdate...
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 184.3/184.3 kB 2.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.

# 3d to 2d slices

1. Converting 3d image to 2d slices.

In [9]:


grid_size = [600, 600, 1000]
grid_res = 0.1
output_dir = '../../data/'
# config_dict = {'raster_layers': ['vertical_density', 'canopy_height'], 'support_window': [11, 7, 1],
#                'normalisation': 'rescale+histeq','doHisteq':[False,True],'res': 0.1, 'gridSize': }

config_dict = {"doHisteq": [False, True], "raster_layers": ["vertical_density", "canopy_height"], "gridSize": grid_size, "res": grid_res, "support_window": [11, 7, 1], "normalisation": "rescale+histeq"}

rasterMaker = treeDetector.RasterDetector(**config_dict)


# PCD file
path = "/forest_3d_app/3d_forest/may22_194/169_254_113_214_pcap/stations/400_1-560_1_slam.pcd"
pcd_file = read_pcd(path)
xyz_data = pcd_file['points'][['x','y','z']].values
offset = [0,0,0]




box_store = np.zeros((1, 4))
counter = 0
windowSize = [50,50]
stepSize = 40

xyz_data_gr = ground_removal.removeGround(xyz_data,offset,thresh=2.0,proc_path=output_dir)
ground_pts = ground_removal.load_ground_surface(os.path.join(output_dir,'_ground_surface.ply'))

xyz_clr_data = xyz_data
slices = detection_tools.sliding_window_3d(xyz_clr_data, stepSize=stepSize,windowSize=windowSize)
for (x, y, window) in slices:  # stepsize 100

    # track progress
    counter = counter 
    totalCount = len(range(int(np.min(xyz_data[:, 0])), int(np.max(xyz_data[:, 0])),stepSize)) * \
                 len(range(int(np.min(xyz_data[:, 1])), int(np.max(xyz_data[:, 1])),stepSize))
    sys.stdout.write("\r%d out of %d tiles" % (counter, totalCount))
    sys.stdout.flush()


0 out of 275 tiles

# Getting Slices 1 by 1

In [3]:
slices = detection_tools.sliding_window_3d(xyz_clr_data, stepSize=stepSize,windowSize=windowSize)


In [4]:
# print(slices)

while  True:
    x, y, window = next(slices)
    if window is not None: break
image, center =rasterMaker._rasterise(window, ground_pts=ground_pts)
image = np.uint8(image*255)
plt.imshow(image)
plt.show()

# Predict Yolov5


In [5]:
import torch
weights = "/forest_3d_app/yolov5/runs/train/exp7/weights/best.pt"

model = torch.hub.load('ultralytics/yolov5', 'custom', weights)
def convert_to_percentage(box, total_width, total_height):
    x, y, w, h = box 
    x_percent = x / total_width
    y_percent = y / total_height
    w_percent = w / total_width
    h_percent = h / total_height

    return np.asarray([x_percent, y_percent, w_percent, h_percent])


def xywh_to_x1y1x3y2(box):
    centerX, centerY, width, height =box
    return [centerY-(height/2.0),centerX-(width/2.0),centerY+(height/2.0),centerX+(width/2.0)]

def detectObjects_yolov5(img):
    
    import matplotlib.pyplot as plt
    import random
    # plt.imsave(f"/forest_3d_app/3d_forest/forest_3d_app/data/test_images/{int(random.random()*100000)}test.png",img)
    # plt.imshow(img)
    # plt.show()
    results = model(img)
    list_bbox = [xywh_to_x1y1x3y2(i) for i in np.asarray(results.pandas().xywh[0][["xcenter", "ycenter", "width","height"]].values)]
    
    
    
    list_conf = np.asarray(results.pandas().xyxy[0]["confidence"].values)
    list_class = np.asarray(results.pandas().xyxy[0]["name"].values)
    list_class = [0 for i in list_class]
    boxes = np.array(list_bbox, np.float64)


    image_width, image_height, channel = img.shape
    percent_boxes = []
    for i in boxes:
        
        percent_boxes.append(convert_to_percentage(i, image_width, image_height))
    return img,percent_boxes,np.array(list_class),np.array(list_conf)



Using cache found in /root/.cache/torch/hub/ultralytics_yolov5_master
YOLOv5 🚀 2023-7-7 Python-3.9.16 torch-2.0.1+cu117 CUDA:0 (NVIDIA GeForce RTX 3070, 7979MiB)

Fusing layers... 
Model summary: 157 layers, 7012822 parameters, 0 gradients
Adding AutoShape... 


# Yolov5 to 3d Boxes

In [6]:
boxesV5 = detectObjects_yolov5(image)
boxesV5 = np.asarray(boxesV5[1])
boxesV5

bb_coord = detection_tools.boundingBox_to_3dcoords(boxes_=boxesV5, gridSize_=grid_size[0:2], gridRes_=grid_res,
                                                                      windowSize_=windowSize, pcdCenter_=center)
print(bb_coord)

with open("/forest_3d_app/3d_forest/forest_3d_app/data/150_2-160_2_slam.txt", "w") as f:
    np.savetxt(f, bb_coord)

type(bb_coord)


IndexError: too many indices for array: array is 1-dimensional, but 2 were indexed

# Perdict full PCD file

In [10]:
boxes= []


slices = detection_tools.sliding_window_3d(xyz_clr_data, stepSize=stepSize,windowSize=windowSize)

iterator = 0
while  True:
    
    x, y, window = next(slices)
    if window is not None:
        image, center =rasterMaker._rasterise(window, ground_pts=ground_pts)
        image = np.uint8(image*255)
        plt.imshow(image)
        plt.show()


        # Predicting
        boxesV5 = np.asarray(detectObjects_yolov5(image)[1])
        print(boxesV5)
        if len(boxesV5):

            bb_coord = detection_tools.boundingBox_to_3dcoords(boxes_=boxesV5, gridSize_=grid_size[0:2], gridRes_=grid_res,
                                                                                  windowSize_=windowSize, pcdCenter_=center)
            boxes.append(bb_coord)
            iterator += 1
            
            



[]
[]
[]
[]
[[    0.16866     0.58741     0.23761     0.61574]
 [    0.32337     0.46336     0.48307     0.52284]
 [    0.33022     0.41903     0.37655     0.46717]
 [    0.27302     0.58302     0.35197     0.61359]]
[[     0.5797     0.48411     0.87176     0.79861]
 [    0.12253     0.46053     0.54654     0.71633]]
[[    0.46215     0.40903     0.64749     0.54559]]
[]
[]
[]
[[    0.57084    0.022469     0.67738     0.11401]
 [    0.68538     0.25088     0.87156     0.63346]]
[[   0.095811      0.4965     0.43389      0.7722]
 [    0.43718     0.51967     0.71402     0.92023]
 [     0.7272     0.65542     0.88046     0.90725]]
[[    0.41835     0.40329     0.58873     0.53121]
 [    0.60244     0.40596     0.70577     0.50133]
 [    0.62237     0.41072     0.81442     0.59629]]
[]
[]
[]
[]
[]
[]
[[    0.40006    0.034676     0.49638    0.077708]
 [    0.32985     0.26695     0.63083     0.63517]]
[[     0.6818     0.64067      0.9989     0.86695]
 [    0.15225     0.66514     0.2681

[[    0.10658     0.51733     0.39488     0.75917]
 [    0.35627     0.50815     0.56301      0.7907]
 [    0.10349     0.47586      0.1644     0.57293]
 [    0.49087     0.37036     0.83604     0.62679]]
[]
[]
[]
[[     0.6103     0.30086     0.77402     0.46739]
 [   0.023428     0.68778     0.15142     0.77912]]
[[    0.23654     0.06318     0.29916     0.10947]
 [    0.89503     0.77778     0.96768     0.90991]
 [    0.53351     0.72389     0.57459     0.77437]
 [     0.1226      0.7751     0.17357     0.85152]
 [    0.13547     0.24489     0.18756     0.30745]
 [    0.30799     0.23361     0.33534      0.2634]]
[[    0.75404     0.50883     0.99805     0.72246]
 [    0.44388     0.63223     0.61696     0.83225]
 [    0.92228     0.34805     0.99108     0.48416]
 [    0.14271     0.35097      0.2027     0.43068]
 [     0.5533     0.30392     0.59686     0.34531]
 [    0.14107     0.45259     0.22698     0.58455]]
[]
[]
[]
[]
[[    0.15024     0.19919     0.37745     0.36856]
 [    

StopIteration: 

In [11]:
flattened_list = [item for sublist in boxes for item in sublist]
npboxes = np.asarray(flattened_list)
with open("/forest_3d_app/3d_forest/forest_3d_app/data/150_2-160_2_slam.txt", "w") as f:
    np.savetxt(f, npboxes)

# type(bb_coord)


In [ ]:
word = "hello I am here Sohaib's"

In [ ]:

title_except_s(word)

In [1]:
"hello".center(40)

'                 hello                  '